# 1. The Lists of Data Table
### 1) Case Data
- **Case**: Data of COVID-19 infection cases in South Korea

### 2) Patient Data
- **PatientInfo**: Epidemiological data of COVID-19 patients in South Korea
- **PatientRoute**: Route data of COVID-19 patients in South Korea

### 3) Time Series Data
- **Time**: Time series data of COVID-19 status in South Korea
- **TimeAge**: Time series data of COVID-19 status in terms of the age in South Korea
- **TimeGender**: Time series data of COVID-19 status in terms of the gender in South Korea
- **TimeProvince**: Time series data of COVID-19 status in terms of the Province in South Korea

### 4) Additional Data
- **Region**: Location and statistical data of the regions in South Korea
- **Weather**: Data of the weather in the regions of South Korea
- **SearchTrend**: Trend data of the keywords searched in NAVER which is one of the largest portal in South Korea

# 2. The Structure of our Dataset
- What color means is that they have similar properties.
- If a line is connected between columns, it means that the values of the columns are partially shared.
- The dotted lines mean weak relevance.
![db](https://user-images.githubusercontent.com/50820635/76959778-a4718700-695d-11ea-864c-379c2c9b97a6.PNG)

# 3. The Detailed Description of each Data Table

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/coronavirusdataset/TimeAge.csv
/kaggle/input/coronavirusdataset/Region.csv
/kaggle/input/coronavirusdataset/Time.csv
/kaggle/input/coronavirusdataset/Weather.csv
/kaggle/input/coronavirusdataset/SearchTrend.csv
/kaggle/input/coronavirusdataset/TimeProvince.csv
/kaggle/input/coronavirusdataset/TimeGender.csv
/kaggle/input/coronavirusdataset/PatientInfo.csv
/kaggle/input/coronavirusdataset/PatientRoute.csv
/kaggle/input/coronavirusdataset/Case.csv


In [2]:
path = '/kaggle/input/coronavirusdataset/'

case = p_info = pd.read_csv(path+'Case.csv')
p_info = pd.read_csv(path+'PatientInfo.csv')
p_route = pd.read_csv(path+'PatientRoute.csv')
time = pd.read_csv(path+'Time.csv')
t_age = pd.read_csv(path+'TimeAge.csv')
t_gender = pd.read_csv(path+'TimeGender.csv')
t_provin = pd.read_csv(path+'TimeProvince.csv')
region = pd.read_csv(path+'Region.csv')
weather = pd.read_csv(path+'Weather.csv')
search = pd.read_csv(path+'SearchTrend.csv')

##### Before the Start..
- We make a structured dataset based on the report materials of KCDC and local governments.
- In Korea, we use the terms named '-do', '-si', '-gun' and '-gu',
- The meaning of them are explained below.

***


### Levels of administrative divisions in South Korea
#### Upper Level (Provincial-level divisions)
- **Special City**:
*Seoul*
- **Metropolitan City**:
*Busan / Daegu / Daejeon / Gwangju / Incheon / Ulsan*
- **Province(-do)**:
*Gyeonggi-do / Gangwon-do / Chungcheongbuk-do / Chungcheongnam-do / Jeollabuk-do / Jeollanam-do / Gyeongsangbuk-do / Gyeongsangnam-do*

#### Lower Level (Municipal-level divisions)
- **City(-si)**
[List of cities in South Korea](https://en.wikipedia.org/wiki/List_of_cities_in_South_Korea)
- **Country(-gun)**
[List of counties of South Korea](https://en.wikipedia.org/wiki/List_of_counties_of_South_Korea)
- **District(-gu)**
[List of districts in South Korea](https://en.wikipedia.org/wiki/List_of_districts_in_South_Korea)

***

![inbox](https://user-images.githubusercontent.com/50820635/76988625-57ef7100-6988-11ea-8276-af4da70d1926.jpg)

***

Sources
- http://nationalatlas.ngii.go.kr/pages/page_1266.php
- https://en.wikipedia.org/wiki/Administrative_divisions_of_South_Korea

### 1) Case
#### Data of COVID-19 infection cases in South Korea
1. case_id: the ID of the infection case
  > - case_id(7) = region_code(5) + case_number(2)  
  > - You can check the region_code in 'Region.csv'
- province: Special City / Metropolitan City / Province(-do)
- city: City(-si) / Country (-gun) / District (-gu)
  > - The value 'from other city' means that where the group infection started is other city.
- group: TRUE: group infection / FALSE: not group
  > - If the value is 'TRUE' in this column, the value of 'infection_cases' means the name of group.  
  > - The values named 'contact with patient', 'overseas inflow' and 'etc' are not group infection. 
- infection_case: the infection case (the name of group or other cases)
  > - The value 'overseas inflow' means that the infection is from other country.  
  > - Tha value 'etc' includes individual cases, cases where relevance classification is ongoing after investigation, and cases under investigation.
- confirmed: the accumulated number of the confirmed
- latitude: the latitude of the group (WGS84)
- longitude: the longitude of the group (WGS84)


In [3]:
case.head()

,case_id,province,city,group,infection_case,confirmed,latitude,longitude
0,1000001,Seoul,Guro-gu,True,Guro-gu Call Center,79,37.508163,126.884387
1,1000002,Seoul,Dongdaemun-gu,True,Dongan Church,24,37.592888,127.056766
2,1000003,Seoul,Eunpyeong-gu,True,Eunpyeong St. Mary's Hospital,14,37.63369,126.9165
3,1000004,Seoul,Seongdong-gu,True,Seongdong-gu APT,13,37.55713,127.0403
4,1000005,Seoul,Jongno-gu,True,Jongno Community Center,10,37.57681,127.006


### 2) PatientInfo
#### Epidemiological data of COVID-19 patients in South Korea
1. patient_id: the ID of the patient
  > - patient_id(10) = region_code(5) + patient_number(5)
  > - You can check the region_code in 'Region.csv'
  > - There are two types of the patient_number  
      1) local_num: The number given by the local government.  
      2) global_num: The number given by the KCDC  
- global_num: the number given by KCDC
  > - There are some patients having no global_num.
  > - The paitents in Busan doesn't have the global_num.
- sex: the sex of the patient
- birth_year: the birth year of the patient
- age: the age of the patient
  > - 0s: 0 ~ 9  
  > - 10s: 10 ~ 19  
  ...  
  > - 90s: 90 ~ 99  
  > - 100s: 100 ~ 109
- country: the country of the patient
- province: the province of the patient
- city: the city of the patient
- disease: TRUE: underlying disease / FALSE: no disease
- infection_case: the case of infection
- infection_order: the order of infection
- infected_by: the ID of who infected the patient
  > - This column refers to the  'patient_id' column. 
- contact_number: the number of contacts with people
- symptom_onset_date: the date of symptom onset
- confirmed_date: the date of being confirmed
- released_date: the date of being released
- deceased_date: the date of being deceased
- state: isolated / released / deceased
  > - isolated: being isolated in the hospital
  > - released: being released from the hospital
  > - deceased: being deceased

In [4]:
p_info.head()

,patient_id,global_num,sex,birth_year,age,country,province,city,disease,infection_case,infection_order,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
0,1000000001,2,male,1964.0,50s,Korea,Seoul,Gangseo-gu,NaN,overseas inflow,1.0,NaN,75.0,2020-01-22,2020-01-23,2020-02-05,NaN,released
1,1000000002,5,male,1987.0,30s,Korea,Seoul,Jungnang-gu,NaN,overseas inflow,1.0,NaN,31.0,NaN,2020-01-30,2020-03-02,NaN,released
2,1000000003,6,male,1964.0,50s,Korea,Seoul,Jongno-gu,NaN,contact with patient,2.0,2002000001,17.0,NaN,2020-01-30,2020-02-19,NaN,released
3,1000000004,7,male,1991.0,20s,Korea,Seoul,Mapo-gu,NaN,overseas inflow,1.0,NaN,9.0,2020-01-26,2020-01-30,2020-02-15,NaN,released
4,1000000005,9,female,1992.0,20s,Korea,Seoul,Seongbuk-gu,NaN,contact with patient,2.0,1000000002,2.0,NaN,2020-01-31,2020-02-24,NaN,released


### 3) PatientRoute
#### Route data of COVID-19 patients in South Korea
- patient_id: the ID of the patient
- global_num: the number given by KCDC
- date: YYYY-MM-DD
- province: Special City / Metropolitan City / Province(-do)
- city: City(-si) / Country (-gun) / District (-gu)
- latitude: the latitude of the visit (WGS84)
- longitude: the longitude of the visit (WGS84)

In [5]:
p_route.head()

,patient_id,global_num,date,province,city,latitude,longitude
0,1400000001,1,2020-01-19,Incheon,Jung-gu,37.460459,126.440680
1,1400000001,1,2020-01-20,Incheon,Seo-gu,37.478832,126.668558
2,1000000001,2,2020-01-22,Gyeonggi-do,Gimpo-si,37.562143,126.801884
3,1000000001,2,2020-01-23,Seoul,Jung-gu,37.567454,127.005627
4,2000000001,3,2020-01-20,Incheon,Jung-gu,37.460459,126.440680


### 4) Time
#### Time series data of COVID-19 status in South Korea
- date: YYYY-MM-DD
- time: Time (0 = AM 12:00 / 16 = PM 04:00)
  > - The time for KCDC to open the information has been changed from PM 04:00 to AM 12:00 since March 2nd.
- test: the accumulated number of tests
  > - A test is a diagnosis of an infection.
- negative: the accumulated number of negative results
- confirmed: the accumulated number of positive results
- released: the accumulated number of releases
- deceased: the accumulated number of deceases

In [6]:
time.head()

,date,time,test,negative,confirmed,released,deceased
0,2020-01-20,16,1,0,1,0,0
1,2020-01-21,16,1,0,1,0,0
2,2020-01-22,16,4,3,1,0,0
3,2020-01-23,16,22,21,1,0,0
4,2020-01-24,16,27,25,2,0,0


### 5) TimeAge
#### Time series data of COVID-19 status in terms of the age in South Korea
- date: YYYY-MM-DD
  > - The status in terms of the age has been presented since March 2nd.
- time: Time
- age: the age of patients
- confirmed: the accumulated number of the confirmed
- deceased: the accumulated number of the deceased

In [7]:
t_age.head()

,date,time,age,confirmed,deceased
0,2020-03-02,0,0s,32,0
1,2020-03-02,0,10s,169,0
2,2020-03-02,0,20s,1235,0
3,2020-03-02,0,30s,506,1
4,2020-03-02,0,40s,633,1


### 6) TimeGender
#### Time series data of COVID-19 status in terms of the gender in South Korea
- date: YYYY-MM-DD
  > - The status in terms of the gender has been presented since March 2nd.
- time: Time
- sex: the gender of patients
- confirmed: the accumulated number of the confirmed
- deceased: the accumulated number of the deceased

In [8]:
t_gender.head()

,date,time,sex,confirmed,deceased
0,2020-03-02,0,male,1591,13
1,2020-03-02,0,female,2621,9
2,2020-03-03,0,male,1810,16
3,2020-03-03,0,female,3002,12
4,2020-03-04,0,male,1996,20


### 7) TimeProvince
#### Time series data of COVID-19 status in terms of the Province in South Korea
- date: YYYY-MM-DD
- time: Time
- province: the province of South Korea
- confirmed: the accumulated number of the confirmed in the province
  > - The confirmed status in terms of the provinces has been presented since Feburary 21th.
  > - The value before Feburary 21th can be different.
- released: the accumulated number of the released in the province
  > - The confirmed status in terms of the provinces has been presented since March 5th.
  > - The value before March 5th can be different.
- deceased: the accumulated number of the deceased in the province
  > - The confirmed status in terms of the provinces has been presented since March 5th.
  > - The value before March 5th can be different.

In [9]:
t_provin.head()

,date,time,province,confirmed,released,deceased
0,2020-01-20,16,Seoul,0,0,0
1,2020-01-20,16,Busan,0,0,0
2,2020-01-20,16,Daegu,0,0,0
3,2020-01-20,16,Incheon,1,0,0
4,2020-01-20,16,Gwangju,0,0,0


### 8) Region
#### Location and statistical data of the regions in South Korea
- code: the code of the region
- province: Special City / Metropolitan City / Province(-do)
- city: City(-si) / Country (-gun) / District (-gu)
- latitude: the latitude of the visit (WGS84)
- longitude: the longitude of the visit (WGS84)
- elementary_school_count: the number of elementary schools
- kindergarten_count: the number of kindergartens
- university_count: the number of universities
- academy_ratio: the ratio of academies
- elderly_population_ratio: the ratio of the elderly population
- elderly_alone_ratio: the ratio of elderly households living alone
- nursing_home_count: the number of nursing homes

Source of the statistic: [KOSTAT (Statistics Korea)](http://kosis.kr/)

In [10]:
region.head()

,code,province,city,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count
0,10000,Seoul,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739
1,10010,Seoul,Gangnam-gu,37.518421,127.047222,33,38,0,4.18,13.17,4.3,3088
2,10020,Seoul,Gangdong-gu,37.530492,127.123837,27,32,0,1.54,14.55,5.4,1023
3,10030,Seoul,Gangbuk-gu,37.639938,127.025508,14,21,0,0.67,19.49,8.5,628
4,10040,Seoul,Gangseo-gu,37.551166,126.849506,36,56,1,1.17,14.39,5.7,1080


### 9) Weather
#### Data of the weather in the regions of South Korea
- code: the code of the region
- province: Special City / Metropolitan City / Province(-do)
- date: YYYY-MM-DD
- avg_temp: the average temperature
- min_temp: the lowest temperature
- max_temp: the highest temperature
- precipitation: the daily precipitation
- max_wind_speed: the maximum wind speed
- most_wind_direction: the most frequent wind direction
- avg_relative_humidity: the average relative humidity

Source of the weather data: [KMA (Korea Meteorological Administration)](http://data.kma.go.kr)

In [11]:
weather.head()

,code,province,date,avg_temp,min_temp,max_temp,precipitation,max_wind_speed,most_wind_direction,avg_relative_humidity
0,10000,Seoul,2016-03-17,9.3,2.0,14.7,0.0,3.2,70.0,40.9
1,10000,Seoul,2016-03-18,13.4,8.8,20.1,0.0,5.0,110.0,50.0
2,10000,Seoul,2016-03-19,11.4,6.6,18.1,0.0,4.8,320.0,58.3
3,10000,Seoul,2016-03-20,10.6,4.9,17.5,0.0,4.6,320.0,54.0
4,10000,Seoul,2016-03-21,9.9,5.5,16.1,0.0,5.4,320.0,54.1


### 10) SearchTrend
#### Trend data of the keywords searched in NAVER which is one of the largest portal in South Korea
- date: YYYY-MM-DD
- cold: the search volume of 'cold' in Korean language
  > - The unit means relative value by setting the highest search volume in the period to 100.
- flu: the search volume of 'flu' in Korean language
  > - Same as above.
- pneumonia: the search volume of 'pneumonia' in Korean language
  > - Same as above.
- coronavirus: the search volume of 'coronavirus' in Korean language
  > - Same as above.


Source of the data: [NAVER DataLab](https://datalab.naver.com/)

In [12]:
search.head()

,date,cold,flu,pneumonia,coronavirus
0,2016-03-17,0.15554,0.34471,0.18181,0.01236
1,2016-03-18,0.14417,0.49416,0.17563,0.01027
2,2016-03-19,0.13290,0.39907,0.15145,0.01154
3,2016-03-20,0.13863,0.39662,0.14408,0.00945
4,2016-03-21,0.16735,0.56989,0.21199,0.00963
